## 人間計算可能なパスワードの機械学習による予測 関数と予測モデルを増やした上での実験

### 目的

村田さんの論文では、多層パーセプトロンを使った人間計算可能なパスワードの攻撃について、複雑な関数では正解率が10%程度となり難しいと結論づけている。また、村田さんの論文では簡単な関数では70%程度正解できると結論づけられている。さらに、丸野さんの論文ではLSTMを用いた予測でも同様に10%程度となり難しいと結論づけている。これに対して、複数の機械学習モデルおよび関数の組み合わせを網羅的に調べて、人間計算可能なパスワードの機械学習による予測について調べる。

### 方法

複数の関数、および機械学習モデルを用意し、それぞれに対してすべての組み合わせで実行させる。

### 今回試す関数

今回は次のような関数を試す。

- 単純な足し算関数
- 合成関数

これらの関数は、村田さんの論文ではf sigmaおよびgという関数として提案されているものである。

- 四重の合成関数

この関数は今回新しく試す関数である。上で試した一層の合成関数と違い、四層の合成関数を用いる。

### 今回試す機械学習モデル

今回の実験では次のモデルを試す。

- MLPモデル
- LSTMモデル(RMSPropで最適化したもの)
- LSTMモデル(Adamで最適化したもの)

ここで、LSTMモデルは二つの最適化関数を用いて試す。これは、どちらかの最適化関数を使った場合に局所解にトラップされ学習が進まなくなるリスクを避けるためのものである。

### 実験のコード

今回の実験は次のコードで実行した。

utils.py
```
from matplotlib import pyplot
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

class Utils:
  # グラフのプロットを行う関数
  # keras.model.fit.historyおよび画像の保存名を引数として取る
  @staticmethod
  def plot_history(history :str, name :str) -> None:
    # 学習時の学習データおよび正解データに対する正解率を表示する
    pyplot.clf()
    pyplot.plot(history['accuracy'])
    pyplot.plot(history['val_accuracy'])
    pyplot.title('model accuracy')
    pyplot.xlabel('epoch')
    pyplot.ylabel('accuracy')
    pyplot.legend(['acc', 'val_acc'], loc='lower right')
    pyplot.savefig("outputs/" + name + '_accuracy.png')

    # 学習時の学習データおよび正解データに対する損失関数の値を表示する
    pyplot.clf()
    pyplot.plot(history['loss'])
    pyplot.plot(history['val_loss'])
    pyplot.title('model loss')
    pyplot.xlabel('epoch')
    pyplot.ylabel('loss')
    pyplot.legend(['loss', 'val_loss'], loc='lower right')
    pyplot.savefig("outputs/" + name + '_loss.png')
    return None

  @staticmethod
  def split_to_train_and_valid(generated_passwords :pd.DataFrame) -> (np.ndarray, np.ndarray, np.ndarray, np.ndarray):
    generated_passwords = generated_passwords.sample(frac=1).reset_index(drop=True)
    x = generated_passwords.drop(labels = ["Z"],axis = 1)
    y = generated_passwords["Z"]

    # 説明変数・目的変数をそれぞれ訓練データ・テストデータに分割
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

    # データの型変換
    #x_train = x_train.astype(float)
    #x_test = x_test.astype(float)

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    return x_train, x_test, y_train, y_test
```

computable_password_generator.py
```
import numpy as np
import pandas as pd

class ComputablePasswordGenerator:
  # 人間計算可能なパスワードの流出データを模したデータを自動生成する関数群
  # このクラスの関数を実行すると、人間計算可能なパスワードを模したデータが生成され、csvファイルとして保存される
  # 外部のプログラムから呼び出すときは、from computable_password_generator import ComputablePasswordGenerator としてimportを行う

  # この関数群の呼び出しには、int: データ数を引数として与える
  # この関数群はpandas.DataFrameをreturnする
  class Utils:
    @staticmethod
    def sgm(n :int) -> np.ndarray:
      sgm = np.random.randint(0,10,n)
      return sgm

  class GeneratorWithMetadata:
    def __init__(self, generator, name :str):
      self.generator = generator
      self.name = name

  @staticmethod
  def list_generators() -> list:
    generators = []
    generators.append(ComputablePasswordGenerator.GeneratorWithMetadata(ComputablePasswordGenerator.simple_pointer, "simple_pointer"))
    generators.append(ComputablePasswordGenerator.GeneratorWithMetadata(ComputablePasswordGenerator.password_with_middle, "middle"))
    generators.append(ComputablePasswordGenerator.GeneratorWithMetadata(ComputablePasswordGenerator.password_simple_add, "simple_add"))
    return generators

  @staticmethod
  def password_simple_add(datasize :int) -> np.ndarray:
    result = []
    for row in range(datasize):
      X = ComputablePasswordGenerator.Utils.sgm(14)
      Z = (X[0] + X[1] + X[2]) % 10
      row = np.append(X, Z)
      result.append(row)
    table_array = np.array(result)
    return pd.DataFrame(table_array, columns=["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X7",
      "X8", "X9", "X10", "X11", "X12","X13", "Z"])

  @staticmethod
  def password_with_middle(datasize :int) -> np.ndarray:
    result = []
    for row in range(datasize):
      X = ComputablePasswordGenerator.Utils.sgm(14)
      mid = (X[10] + X[11]) % 10
      Z = (X[mid] + X[12]) % 10
      row = np.append(X, Z)
      result.append(row)
    table_array = np.array(result)
    return pd.DataFrame(table_array, columns=["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X7",
      "X8", "X9", "X10", "X11", "X12","X13", "Z"])

  @staticmethod
  def simple_pointer(datasize :int) -> np.ndarray:
    result = []
    for row in range(datasize):
      X = ComputablePasswordGenerator.Utils.sgm(14)
      mid_1 = (X[10] + X[11]) % 10
      mid_2 = (mid_1 + X[13]) % 10
      mid_3 = (mid_2 + X[12]) % 10
      Z = [mid_3] + X[12]
      row = np.append(X, Z)
      result.append(row)
    table_array = np.array(result)
    return pd.DataFrame(table_array, columns=["X0", "X1", "X2", "X3", "X4", "X5", "X6", "X7",
      "X8", "X9", "X10", "X11", "X12","X13", "Z"])
```

models.py
```
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.layers import Flatten
import pandas as pd
import numpy as np

class Models:
  class ModelWithMetadata:
    def __init__(self, model, name :str, batch_size :int, epochs :int, required_data_size :int):
      self.model = model
      self.name = name
      self.batch_size = batch_size
      self.epochs = epochs
      self.required_data_size = required_data_size

    def resharper(self, df :pd.DataFrame) -> (np.ndarray):
      if self.name.find("lstm") != -1:
        print("hoge")
        return df.to_numpy().reshape(df.shape[0], df.shape[1], 1)
      return df

  @staticmethod
  def list_models() -> list:
    models = []
    models.append(Models.ModelWithMetadata(Models.mlp_model(), "mlp", 16, 512, 10000))
    models.append(Models.ModelWithMetadata(Models.simple_lstm_with_AMSGrad(), "simple_lstm", 32, 1024, 50000))
    models.append(Models.ModelWithMetadata(Models.simple_lstm_with_adam(), "lstm_with_adam", 32, 1024, 50000))
    return models

  # 人間計算可能なパスワードの予測に使うための機械学習モデル群
  # これらの関数を呼び出すと、指定したSequentialモデルがreturnされる
  @staticmethod
  def mlp_model() -> Sequential:
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

  @staticmethod
  def simple_lstm_with_AMSGrad() -> Sequential:
    model = Sequential()
    model.add(LSTM(32, input_shape = (14, 1)))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="mean_squared_error", optimizer="RMSprop", metrics=["accuracy"])
    return model

  @staticmethod
  def simple_lstm_with_adam() -> Sequential:
    model = Sequential()
    model.add(LSTM(32, input_shape = (14, 1)))
    model.add(Dense(10, activation="softmax"))
    model.compile(loss="mean_squared_error", optimizer="Adam", metrics=["accuracy"])
    return model
```

main.py
```
from computable_password_generator import ComputablePasswordGenerator
from utils import Utils
from models import Models

for model in Models.list_models():
  print("Runnning model: {}".format(model.name))
  for generator in ComputablePasswordGenerator.list_generators():
    try:
      print("Testing: generator: {}, model: {}".format(generator.name, model.name))
      print("Figure name: {}".format(generator.name + "_" + model.name))
      generated_passwords = generator.generator(model.required_data_size)
      x_train, x_test, y_train, y_test = Utils.split_to_train_and_valid(generated_passwords)
      x_train = model.resharper(x_train)
      print(x_train.shape)
      x_test = model.resharper(x_test)
      model.model.fit(x_train,y_train,batch_size=model.batch_size, epochs=model.epochs, verbose=1, validation_data=(x_test, y_test))
      history = model.model.history.history
      Utils.plot_history(history, generator.name + "_" + model.name)
    except Exception as e:
      print("Error: generator: {}, model: {}".format(generator.name, model.name))
      print(e)
      continue
```


### 実験結果

実験の結果生成された損失関数のグラフおよび正解率のグラフは次のようになった。

単純な足し算関数(simple_add)の場合

MLPモデルの場合

![simple_add_mlp_loss](../outputs/simple_add_mlp_loss.png)
![simple_add_mlp_accuracy](../outputs/simple_add_mlp_accuracy.png)

LSTMモデル(最適化関数: RMSProp)の場合

![simple_add_lstm_loss](../outputs/simple_add_simple_lstm_loss.png)
![simple_add_lstm_accuracy](../outputs/simple_add_simple_lstm_accuracy.png)

LSTMモデル(最適化関数: Adam)の場合

![simple_add_lstm_with_adam_loss](../outputs/simple_add_lstm_with_adam_loss.png)
![simple_add_lstm_with_adam_accuracy](../outputs/simple_add_lstm_with_adam_accuracy.png)

合成関数(middle)の場合

MLPモデルの場合

![middle_mlp_loss](../outputs/middle_mlp_loss.png)
![middle_mlp_accuracy](../outputs/middle_mlp_accuracy.png)

LSTMモデル(最適化関数: RMSProp)の場合

![middle_lstm_loss](../outputs/middle_simple_lstm_loss.png)
![middle_lstm_accuracy](../outputs/middle_simple_lstm_accuracy.png)

LSTMモデル(最適化関数: Adam)の場合

![middle_lstm_with_adam_loss](../outputs/middle_lstm_with_adam_loss.png)
![middle_lstm_with_adam_accuracy](../outputs/middle_lstm_with_adam_accuracy.png)

四重合成関数(pointer)の場合

MLPモデルの場合

![simple_pointer_mlp_loss](../outputs/simple_pointer_mlp_loss.png)
![simple_pointer_mlp_accuracy](../outputs/simple_pointer_mlp_accuracy.png)

LSTMモデル(最適化関数: RMSProp)の場合

![simple_pointer_lstm_loss](../outputs/simple_pointer_simple_lstm_loss.png)
![simple_pointer_lstm_accuracy](../outputs/simple_pointer_simple_lstm_accuracy.png)

LSTMモデル(最適化関数: Adam)の場合

![simple_pointer_lstm_with_adam_loss](../outputs/simple_pointer_lstm_with_adam_loss.png)
![simple_pointer_lstm_with_adam_accuracy](../outputs/simple_pointer_lstm_with_adam_accuracy.png)

### 考察

これまでの論文で検証されていた組み合わせについては、これまでの論文とほぼ同じ結果になったと言える。ここで、同じLSTMモデルでも最初の足し算関数については最適化関数によってグラフの形が大きく変わっていることがわかる。これは、RMSPropが学習率を適宜調整する仕組みであることに対して、AdamはRMSPropの結果にMomentumの結果を加えた分の学習をするという差があることによると考えられる。RMSPropでは最適化がうまくいったが、Adamでは途中で学習率が大きくなりすぎた結果局所解にトラップされた可能性が高いと考えられる。また、中間層が一つだけの関数と比較して、複数の合成関数を活用する関数の場合ではLSTMモデルを活用した場合でも正解率が10%程度となっており、従来の一層だけの合成関数と比較して機械学習による予測が難しくなったといえる。

### 今後の課題

機械学習での予測を難しくするための手法として、複数の合成関数を活用する手法があることがわかった。一方で、人間にとっては扱いづらいパスワードとなることが予想される。そのため、どこまで安全性を担保する必要が現実的にあるのかを調べるために、従来の関数に対して様々なモデルで予測可能性の評価を行う必要がある。
